<a href="https://colab.research.google.com/github/CanopySimulations/canopy-python-examples/blob/master/loading_scalar_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Upgrade Runtime
This cell ensures the runtime supports `asyncio` async/await, and is needed on Google Colab. If the runtime is upgraded, you will be prompted to restart it, which you should do before continuing execution.

In [0]:
!pip install "ipython>=7"

# Set Up Environment

### Import required libraries

In [0]:
!pip install -q 'canopy==7.2'

In [0]:
import canopy
import logging

logging.basicConfig(level=logging.INFO)

### Authenticate

In [0]:
authentication_data = canopy.prompt_for_authentication()

def create_session():
    return canopy.Session(authentication_data)

# Set Up Example

Run a quick ApexSim exploration.

In [4]:
async with create_session() as session:
    logging.info('Loading configs...')

    default_car = await canopy.load_default_config(
        session, 'car', 'Canopy F1 Car 2019')
    
    default_weather = await canopy.load_default_config(
        session, 'weather', '25 deg, dry')
    
    default_exploration = await canopy.load_default_config(
        session, 'exploration', 'Automated Test Monte Carlo')

    # We'll run a very small exploration for this example
    default_exploration.data.design.numberOfPoints = 3

    logging.info('Creating study...')

    study_id = await canopy.create_study(
        session,
        'apexSim',
        'Loading Scalar Results Example',
        [
            default_car,
            default_weather,
            default_exploration,
        ])
    
    logging.info('Waiting for study to complete...')

    wait_result = await canopy.wait_for_study(
        session,
        study_id,
        timeout_seconds=300)
    
logging.info('Sim Version: {}'.format(wait_result.document.sim_version))    
logging.info('Succeded Simulations: {}/{}'.format(
    wait_result.succeeded_simulation_count, 
    wait_result.simulation_count))

INFO:root:Loading configs...
INFO:root:Creating study...
INFO:root:Waiting for study to complete...
INFO:root:Sim Version: 1.3043
INFO:root:Succeded Simulations: 3/3


# Example: Loading Scalar Results

## Loading for a single job
We can load the scalar results for an individual job using the `load_study_job` helper function with the `include_scalar_results` parameter. Note we are specifically loading the `ApexSim` scalar results here. If we also wanted the `StraightSim` results we can fetch them in a second call.

In [5]:
async with create_session() as session:
    job_0 = await canopy.load_study_job(
        session,
        study_id,
        'ApexSim',
        job_index=0,
        include_scalar_results=True)
    
job_0.scalar_data

INFO:canopy.load_study_job:Loading job index 0


{'rLatBalF100': 0.591313,
 'rLatBalF200': 0.6492760000000001,
 'rLatBalF300': 0.6833130000000001,
 'kRollCar100': 377345.834901,
 'kRollCar200': 390780.864954,
 'kRollCar300': 395791.790421,
 'rAeroBal100': 0.5235380000000001,
 'rAeroBal200': 0.5174449999999999,
 'rAeroBal300': 0.537024,
 'rAeroBalvMax': 0.552127,
 'rhoAir': 1.182251,
 'hRideF100': 0.021309,
 'hRideF200': 0.009343,
 'hRideF300': -0.008104,
 'hRideR100': 0.138955,
 'hRideR200': 0.12222899999999999,
 'hRideR300': 0.097728,
 'zCarBrakingvMax': -0.302118,
 'aPitchBrakingvMax': -0.035464999999999997,
 'aPitchBrakingvMin': -0.030558999999999996,
 'aRockerFLBrakingvMax': -0.051687000000000004,
 'aRockerFLAcceleratingvMin': 0.0057740000000000005,
 'aRockerRLBrakingvMax': -0.166323,
 'aRockerRLBrakingMax': 0.020674,
 'aRockerRLAcceleratingvMin': -0.027404}

We can request individual scalar results in our desired units using the `scalar_as` function:

In [6]:
job_0.scalar_as('hRideF200', 'mm')

9.343

You can also use the `scalar_as` function and omit the units, which will then use the units from your preferences saved on the Canopy platform:

In [7]:
job_0.scalar_as('hRideF200')

0.009343

## Loading for an entire study

We can load the scalar results for the entire study using the `load_study` helper function with the `include_study_scalar_results` parameter. 

This loads the aggregated scalar results produced by the post-processing job when the study was run, so it is more efficient than loading the scalar results for each study individually.

Note that the aggregated scalar results contain the scalar results for every simulation run in the study (in this case both ApexSim and StraightSim), and so the variable names are of the form `<Name>:<SimType>` to avoid conflicts.

In [8]:
async with create_session() as session:
    study = await canopy.load_study(
        session,
        study_id,
        include_study_scalar_results=True)

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


In [9]:
study.scalar_results.results_metadata

,name,units,description,simType
fullName,,,,
aPitchBrakingvMax:ApexSim,aPitchBrakingvMax,rad,aPitch under straight line braking at vMax.,ApexSim
aPitchBrakingvMin:ApexSim,aPitchBrakingvMin,rad,aPitch under straight line acceleration at vMin.,ApexSim
aRockerFLAcceleratingvMin:ApexSim,aRockerFLAcceleratingvMin,rad,aRockerFL under straight line acceleration at ...,ApexSim
aRockerFLBrakingvMax:ApexSim,aRockerFLBrakingvMax,rad,aRockerFL under straight line braking at vMax.,ApexSim
aRockerRLAcceleratingvMin:ApexSim,aRockerRLAcceleratingvMin,rad,aRockerRL under straight line acceleration at ...,ApexSim
...,...,...,...,...
vCarMaxElectricDRS:StraightSim,vCarMaxElectricDRS,m/s,Maximum vCar achievable on a flat track at wid...,StraightSim
zCarMax:StraightSim,zCarMax,m,Maximum zCar achieved during this simulation.,StraightSim
zCarMin:StraightSim,zCarMin,m,Minimum zCar achieved during this simulation.,StraightSim


In [10]:
study.scalar_results.results

,jobIndex,aPitchBrakingvMax:ApexSim,aPitchBrakingvMin:ApexSim,aRockerFLAcceleratingvMin:ApexSim,aRockerFLBrakingvMax:ApexSim,aRockerRLAcceleratingvMin:ApexSim,aRockerRLBrakingMax:ApexSim,aRockerRLBrakingvMax:ApexSim,hRideF100:ApexSim,hRideF200:ApexSim,hRideF300:ApexSim,hRideR100:ApexSim,hRideR200:ApexSim,hRideR300:ApexSim,kRollCar100:ApexSim,kRollCar200:ApexSim,kRollCar300:ApexSim,rAeroBal100:ApexSim,rAeroBal200:ApexSim,rAeroBal300:ApexSim,rAeroBalvMax:ApexSim,rLatBalF100:ApexSim,rLatBalF200:ApexSim,rLatBalF300:ApexSim,rhoAir:ApexSim,zCarBrakingvMax:ApexSim,tApexSim:Debug,tApexSimScalarResults:Debug,tApexSimVectorResults:Debug,tComponentSweepsVectorResults:Debug,tStraightSim:Debug,tStraightSimScalarResults:Debug,tStraightSimVectorResults:Debug,aCamberTyreF100:StraightSim,aCamberTyreF200:StraightSim,aCamberTyreF300:StraightSim,aCamberTyreFvMax:StraightSim,aCamberTyreR100:StraightSim,aCamberTyreR200:StraightSim,aCamberTyreR300:StraightSim,...,hRideUndertrayFrontvMax:StraightSim,hRideUndertrayMid100:StraightSim,hRideUndertrayMid200:StraightSim,hRideUndertrayMid300:StraightSim,hRideUndertrayMidvMax:StraightSim,hRideUndertrayRear100:StraightSim,hRideUndertrayRear200:StraightSim,hRideUndertrayRear300:StraightSim,hRideUndertrayRearvMax:StraightSim,kRollCar100:StraightSim,kRollCar200:StraightSim,kRollCar300:StraightSim,kRollCarvMax:StraightSim,lPushrodShimFL:StraightSim,lPushrodShimFR:StraightSim,lPushrodShimRL:StraightSim,lPushrodShimRR:StraightSim,lSpringShimFL:StraightSim,lSpringShimFR:StraightSim,lSpringShimRL:StraightSim,lSpringShimRR:StraightSim,rAeroBal100:StraightSim,rAeroBal200:StraightSim,rAeroBal300:StraightSim,rAeroBalvMax:StraightSim,rSlipGripMaxF_2kn:StraightSim,rSlipGripMaxR_2kn:StraightSim,rStiffnessBalF100:StraightSim,rStiffnessBalF200:StraightSim,rStiffnessBalF300:StraightSim,rStiffnessBalFvMax:StraightSim,rhoAir:StraightSim,vCarMax:StraightSim,vCarMaxDRS:StraightSim,vCarMaxElectric:StraightSim,vCarMaxElectricDRS:StraightSim,zCarMax:StraightSim,zCarMin:StraightSim,zHubOffsetF:StraightSim,zHubOffsetR:StraightSim
0,0,-0.035465,-0.030559,0.005774,-0.051687,-0.027404,0.020674,-0.166323,0.021309,0.009343,-0.008104,0.138955,0.122229,0.097728,377345.834901,390780.864954,395791.790421,0.523538,0.517445,0.537024,0.552127,0.591313,0.649276,0.683313,1.182251,-0.302118,6.002493,0.015924,0.877577,0.632749,1.113279,0.012650,0.852302,-0.051825,-0.050423,-0.050068,-0.050053,-0.010713,-0.011639,-0.013469,...,0.002963,0.085886,0.069945,0.047412,0.044393,0.137227,0.118071,0.089580,0.086614,381487.877007,388784.699250,396858.760039,397251.281024,-0.000575,-0.000575,-0.005849,-0.005849,0.0,0.0,0.0,0.0,0.516525,0.503698,0.511529,0.521079,0.118367,0.126531,0.680272,0.697639,0.726737,0.729646,1.182251,78.578958,83.402597,83.255950,87.867760,-0.288308,-0.340116,0.248530,0.266658
1,1,-0.032430,-0.028585,0.005929,-0.052779,-0.027430,0.020926,-0.201112,0.027618,0.016134,0.000134,0.137986,0.120414,0.091469,375547.882567,388440.657633,391054.580420,0.511130,0.496745,0.496180,0.514813,0.591098,0.648351,0.685723,1.182251,-0.299029,6.331092,0.008953,0.525486,0.473990,1.127291,0.008768,0.554414,-0.051796,-0.050303,-0.049875,-0.049852,-0.010719,-0.011711,-0.013676,...,0.009043,0.088145,0.071997,0.048715,0.045164,0.136278,0.116344,0.085654,0.081976,379926.544357,385959.059782,393651.856015,394298.619337,0.001825,0.001825,-0.005544,-0.005544,0.0,0.0,0.0,0.0,0.505032,0.484369,0.478529,0.486394,0.118367,0.126531,0.679733,0.696760,0.727744,0.731190,1.182251,78.927410,83.991803,83.681850,88.651449,-0.288339,-0.342358,0.254677,0.265802
2,2,-0.031346,-0.027938,0.005854,-0.052782,-0.027294,0.020632,-0.208439,0.028441,0.017102,0.001446,0.136296,0.118465,0.088990,374704.111068,387401.933334,389375.784709,0.505886,0.490859,0.488502,0.506079,0.591358,0.648987,0.687308,1.182251,-0.297686,6.590216,0.013976,1.198422,0.485239,1.205162,0.016126,1.141656,-0.051795,-0.050295,-0.049852,-0.049827,-0.010735,-0.011792,-0.013838,...,0.009788,0.087573,0.071356,0.048081,0.04442

You also have access to the study input data as defined by the exploration:

In [11]:
study.scalar_results.inputs_metadata

,units,description,fullName,shortName,modifier
inputName,,,,,
car.chassis.carRunningMass.mCar+,kg,,car.chassis.carRunningMass.mCar,mCar,+
car.chassis.ICar+,kgm2,,car.chassis.ICar,ICar,+
car.chassis.hRideFSetup+,m,,car.chassis.hRideFSetup,hRideFSetup,+
car.chassis.hRideRSetup+,m,,car.chassis.hRideRSetup,hRideRSetup,+


In [12]:
study.scalar_results.inputs

,car.chassis.carRunningMass.mCar+,car.chassis.ICar+,car.chassis.hRideFSetup+,car.chassis.hRideRSetup+
0,2.063011,"[-1.90548291053,0.0,0.0,0.0,-3.81096582106,0.0...",0.002994,0.018050
1,7.295651,"[5.94347604826,0.0,0.0,0.0,11.88695209652,0.0,...",0.009080,0.017123
2,-0.019823,"[-5.02973521364,0.0,0.0,0.0,-10.05947042728,0....",0.009839,0.015512


We can convert this data to our desired units using the `study.scalar_as` helper function. The `scalar_as` function will work for both scalar inputs and results, and if the units are omitted it will use the units from your preferences saved on the Canopy platform.

In [13]:
hRideF200_mm = study.scalar_as('hRideF200:ApexSim', 'mm')
hRideF200_mm

0     9.343
1    16.134
2    17.102
Name: hRideF200:ApexSim, dtype: float64

In [14]:
hRideFSetup_mm = study.scalar_as('car.chassis.hRideFSetup+', 'mm')
hRideFSetup_mm

0    2.993966
1    9.079787
2    9.838959
Name: car.chassis.hRideFSetup+, dtype: float64